In [37]:
import pandas as pd
import numpy as np
import json

In [20]:
def average(series):
    return series.astype(str).apply(lambda x: np.mean(
        [float(n) for n in x.split('-')
    ])).astype(float)


In [38]:
df = pd.read_excel('density_DB_v2_0_final-1__1_.xlsx', sheet_name='Density DB')
df['Density'] = average(df['Density in g/ml (including mass and bulk density)'])
df['Specific gravity'] = average(df['Specific gravity'])
df['Density'].fillna(df['Specific gravity'], inplace=True)
df['Ingredient'] = df['Food name and description'].str.capitalize()
prev_i = None
for i, row in df.iterrows():
    d = pd.isna(df.loc[i, 'Density']) 
    if pd.isna(df.loc[i, 'Density']) or i == len(df) - 1:
        if prev_i is not None:
            df.loc[prev_i, 'Density'] = df['Density'][prev_i+1:i].mean()
        prev_i = i
df

,Food name and description,Density in g/ml (including mass and bulk density),Specific gravity,BiblioID,Update Version 2.0,Density,Ingredient
0,"Beverages, non alcoholic (including soft drink...",NaN,NaN,NaN,NaN,1.033904,"Beverages, non alcoholic (including soft drink..."
1,"Apple juice, canned or bottled, unsweetened, w...",1.043397,1.045725,USDA,x,1.043397,"Apple juice, canned or bottled, unsweetened, w..."
2,"Apple juice, canned or bottled, unsweetened, w...",1.043397,1.045725,USDA,x,1.043397,"Apple juice, canned or bottled, unsweetened, w..."
3,"Beverages, fruit juice drink, reduced sugar, w...",1.043097,1.045425,USDA,x,1.043097,"Beverages, fruit juice drink, reduced sugar, w..."
4,"Carbonated beverage, tonic water",1.021055,1.023333,USDA,x,1.021055,"Carbonated beverage, tonic water"
...,...,...,...,...,...,...,...
660,"Ugali + vegetable, crumbed",0.75,NaN,KEN,x,0.750000,"Ugali + vegetable, crumbed"
661,"Ugali + vegetables, boiled with fat",1.2,NaN,KEN,x,1.200000,"Ugali + vegetables, boiled with fat"
662,"Ugali, plain (flour + water), crumbed",0.75,NaN,KEN,x,0.750000,"Ugali, plain (flour + water), crumbed"
663,Wieners and sauerkraut (mixture),0.612,NaN,S&W,NaN,0.612000,Wieners and sauerkraut (mixture)


In [42]:
densities = {row['Ingredient']: row['Density'] for i, row in df.iterrows()}

with open('../app/src/main/res/raw/densities.json', 'w') as f:
    json.dump(densities, f, indent=4)